# Carga de Datos del PDF en Neo4j

Este notebook carga las entidades y relaciones extraídas del PDF en la base de datos Neo4j.

## Prerequisitos

1. **Neo4j debe estar corriendo**: 
   ```bash
   docker compose up neo4j
   ```

2. **Archivos CSV generados**: 
   - `pdf_extraction_output/entities.csv`
   - `pdf_extraction_output/relations.csv`
   
   (Generados por el notebook `pdf_extraction.ipynb`)

3. **Variables de entorno** configuradas en `.env`:
   - `NEO4J_HOST`
   - `NEO4J_PORT`
   - `NEO4J_USER`
   - `NEO4J_PASSWORD`


## Paso 1: Importar librerías y funciones auxiliares


In [1]:
import csv
import json
import os
from slugify import slugify

from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()

print("✓ Librerías importadas")


✓ Librerías importadas


In [3]:
#clean al data from neo4j
from neo4j import GraphDatabase
import os

host = os.environ.get("NEO4J_HOST", "localhost")
port = os.environ.get("NEO4J_PORT", "7687")
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo4j")

driver = GraphDatabase.driver(f"bolt://{host}:{port}", auth=(user, password))
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
driver.close()
print("✓ Base de datos limpiada")


✓ Base de datos limpiada


### Funciones auxiliares

Funciones para escapar strings, generar URIs y mapear tipos de entidades.


In [11]:
def escape(string):
    """Escapa strings para uso seguro en Cypher"""
    return json.dumps(string)[1:-1]

def generate_uri(name):
    """Genera un URI único (ASCII) pensado para español; preserva nombre original en propiedades."""
    # Normaliza tildes y caracteres españoles con fallback si la versión de slugify no soporta 'language'
    try:
        return slugify(name, separator="_", lowercase=True, language="es")
    except TypeError:
        return slugify(name, separator="_").lower()

# Mapeo de tipos de entidades a labels de Neo4j (incluye etiquetas de spaCy en español)
ENTITY_TYPE_MAPPING = {
    # Modelos en inglés (compatibilidad)
    'PERSON': 'PERSON',
    'ORG': 'ORG',
    'GPE': 'GPE',  # países/ciudades
    'LOC': 'LOC',
    'FAC': 'FAC',
    'PRODUCT': 'PRODUCT',
    'EVENT': 'EVENT',
    'WORK_OF_ART': 'WORK_OF_ART',
    'LAW': 'LAW',
    'LANGUAGE': 'LANGUAGE',
    'DATE': 'DATE',
    'TIME': 'TIME',
    'PERCENT': 'PERCENT',
    'MONEY': 'MONEY',
    'QUANTITY': 'QUANTITY',
    'ORDINAL': 'ORDINAL',
    'CARDINAL': 'CARDINAL',
    # Modelos de spaCy en español
    'PER': 'PERSON',
    'MISC': 'ENTITY'
}

# Mapeo de relaciones comunes a tipos específicos (normalizado a ASCII MAYÚSCULAS con _)
RELATION_TYPE_MAPPING = {
    # Inglés (compatibilidad)
    'is': 'ES',
    'from': 'DESDE',
    'in': 'EN',
    'of': 'DE',
    'to': 'A',
    'with': 'CON',
    'at': 'EN',
    'for': 'PARA',
    'on': 'SOBRE',
    'by': 'POR',

    # Español – preposiciones frecuentes
    'de': 'DE',
    'en': 'EN',
    'a': 'A',
    'con': 'CON',
    'por': 'POR',
    'para': 'PARA',
    'sobre': 'SOBRE',
    'desde': 'DESDE',
    'hasta': 'HASTA',
    'entre': 'ENTRE',
    'sin': 'SIN',
    'según': 'SEGUN',
    'segun': 'SEGUN',
    'contra': 'CONTRA',
    'durante': 'DURANTE',
    'hacia': 'HACIA',
    'mediante': 'MEDIANTE',
    'tras': 'TRAS',
    'bajo': 'BAJO',
    'ante': 'ANTE',

    # Español – verbos/locuciones frecuentes (se transforman a nombres de relación canónicos)
    'es': 'ES',
    'son': 'ES',
    'fue': 'ES',
    'era': 'ES',
    'sera': 'ES',
    'será': 'ES',
    'pertenece a': 'PERTENECE_A',
    'pertenece': 'PERTENECE_A',
    'aparece en': 'APARECE_EN',
    'ubicado en': 'UBICADO_EN',
    'ubicada en': 'UBICADO_EN',
    'nace en': 'NACE_EN',
    'vive en': 'VIVE_EN',
    'parte de': 'PARTE_DE',
    'miembro de': 'MIEMBRO_DE',
    'hijo de': 'HIJO_DE',
    'padre de': 'PADRE_DE',
    'amigo de': 'AMIGO_DE',
    'enemigo de': 'ENEMIGO_DE'
}

print("✓ Funciones auxiliares definidas")


✓ Funciones auxiliares definidas


## Paso 2: Configurar conexión a Neo4j


In [12]:
# Obtener configuración de Neo4j desde variables de entorno
host = os.environ.get("NEO4J_HOST", "localhost")
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "EchoesOfWisdom")
port = os.environ.get("NEO4J_PORT", "7687")

print(f"Configuración Neo4j:")
print(f"  Host: {host}")
print(f"  Puerto: {port}")
print(f"  Usuario: {user}")

# Conectar a Neo4j y verificar conexión inmediatamente
try:
    driver = GraphDatabase.driver(f"bolt://{host}:{port}", auth=(user, password))
    print("\n✓ Driver de Neo4j creado")
    
    # Verificar conexión inmediatamente
    print("Verificando conexión...")
    with driver.session() as session:
        result = session.run("RETURN 1 as test")
        result.single()
    print("✓ Conexión a Neo4j establecida correctamente")
    
except Exception as e:
    print(f"\n✗ Error al conectar a Neo4j: {e}")
    print("\nAsegúrate de que:")
    print("1. Neo4j está corriendo (docker compose up neo4j)")
    print("2. Las credenciales en .env son correctas")
    print("3. El puerto 7687 está accesible")
    raise


Configuración Neo4j:
  Host: localhost
  Puerto: 7687
  Usuario: neo4j

✓ Driver de Neo4j creado
Verificando conexión...
✓ Conexión a Neo4j establecida correctamente


## Paso 3: Verificar datos existentes

Verificamos si ya hay datos del PDF en la base de datos para evitar duplicados.


In [13]:
# Verificar datos existentes del PDF
with driver.session() as session:
    result = session.run("MATCH (n:ENTITY) WHERE n.source = 'pdf' RETURN count(n) as count")
    existing_count = result.single()['count']

if existing_count > 0:
    print(f"\n⚠ Ya existen {existing_count} entidades del PDF en la base de datos")
    print("Si continúas, se agregarán más datos (puede haber duplicados)")
    print("Para empezar desde cero, elimina los datos existentes primero.")
else:
    print(f"\n✓ Base de datos limpia (0 entidades del PDF)")



✓ Base de datos limpia (0 entidades del PDF)


## Paso 4: Verificar archivos CSV

Verificamos que los archivos generados por `pdf_extraction.ipynb` estén disponibles.


In [14]:
output_dir = "pdf_extraction_output"
entities_file = os.path.join(output_dir, "entities.csv")
relations_file = os.path.join(output_dir, "relations.csv")

print("Verificando archivos...")
print("=" * 60)

# Verificar archivo de entidades
if os.path.exists(entities_file):
    file_size = os.path.getsize(entities_file)
    print(f"✓ Entidades encontrado: {entities_file} ({file_size:,} bytes)")
    
    # Contar líneas (aproximado)
    with open(entities_file, 'r', encoding='utf-8') as f:
        line_count = sum(1 for _ in f) - 1  # -1 para el header
    print(f"  → Aproximadamente {line_count} entidades")
else:
    print(f"✗ Error: No se encontró {entities_file}")
    print("Ejecuta primero el notebook pdf_extraction.ipynb para generar los archivos")
    raise FileNotFoundError(f"No se encontró {entities_file}")

# Verificar archivo de relaciones
if os.path.exists(relations_file):
    file_size = os.path.getsize(relations_file)
    print(f"✓ Relaciones encontrado: {relations_file} ({file_size:,} bytes)")
    
    # Contar líneas (aproximado)
    with open(relations_file, 'r', encoding='utf-8') as f:
        line_count = sum(1 for _ in f) - 1  # -1 para el header
    print(f"  → Aproximadamente {line_count} relaciones")
else:
    print(f"⚠ Advertencia: No se encontró {relations_file}")
    relations_file = None

print(f"\n💡 Los archivos pueden contener datos normalizados o no,")
print(f"   según la configuración USE_NORMALIZED_DATA en pdf_extraction.ipynb")


Verificando archivos...
✓ Entidades encontrado: pdf_extraction_output/entities.csv (65,366 bytes)
  → Aproximadamente 2087 entidades
✓ Relaciones encontrado: pdf_extraction_output/relations.csv (12,766,036 bytes)
  → Aproximadamente 9789 relaciones

💡 Los archivos pueden contener datos normalizados o no,
   según la configuración USE_NORMALIZED_DATA en pdf_extraction.ipynb


## Paso 5: Crear índices en Neo4j

Los índices mejoran significativamente el rendimiento de las consultas.


In [15]:
def create_indices(driver):
    """Crea índices para mejorar el rendimiento de las consultas"""
    with driver.session() as session:
        # Índices para entidades
        session.run("CREATE INDEX IF NOT EXISTS FOR (e:ENTITY) ON (e.name)")
        session.run("CREATE INDEX IF NOT EXISTS FOR (e:ENTITY) ON (e.uri)")
        # Índices para otros tipos de nodos
        session.run("CREATE INDEX IF NOT EXISTS FOR (e:PERSON) ON (e.name)")
        session.run("CREATE INDEX IF NOT EXISTS FOR (e:ORG) ON (e.name)")
        session.run("CREATE INDEX IF NOT EXISTS FOR (e:GPE) ON (e.name)")
        session.run("CREATE INDEX IF NOT EXISTS FOR (e:LOC) ON (e.name)")
        print("✓ Índices creados")

print("Creando índices...")
create_indices(driver)


Creando índices...
✓ Índices creados


## Paso 6: Insertar entidades

Cargamos las entidades desde el CSV y las insertamos en Neo4j con sus tipos correspondientes.


In [16]:
def insert_entities(driver, entities_file):
    """Inserta entidades desde el CSV extraído del PDF"""
    entities_inserted = 0
    
    # Query base para crear entidades
    # IMPORTANTE: Los labels NO pueden ser parámetros en Neo4j, 
    # deben construirse dinámicamente en el query
    query_base = """
    MERGE (e:ENTITY {uri: $uri})
    ON CREATE SET 
        e.name = $name,
        e.source = 'pdf',
        e.created_at = timestamp()
    ON MATCH SET
        e.updated_at = timestamp()
    """.strip()

    with driver.session() as session:
        entities_processed = set()  # Para evitar duplicados
        
        with open(entities_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                entity_name = row.get('text', '').strip()
                entity_label = row.get('label', '').strip()
                
                if not entity_name or len(entity_name) < 2:
                    continue
                
                # Generar URI único
                uri = generate_uri(entity_name)
                
                # Evitar duplicados
                entity_key = (uri, entity_label)
                if entity_key in entities_processed:
                    continue
                entities_processed.add(entity_key)
                
                # Usar el nombre directamente (Neo4j escapa automáticamente los parámetros)
                name = entity_name
                
                # Determinar el label de Neo4j
                neo4j_label = ENTITY_TYPE_MAPPING.get(entity_label, 'ENTITY')
                
                # Construir el query completo con el label específico si aplica
                # Los labels no pueden ser parámetros, deben construirse en el query
                if neo4j_label != 'ENTITY' and neo4j_label in ['PERSON', 'ORG', 'GPE', 'LOC', 'FAC']:
                    # Construir query con label específico (ej: SET e:ENTITY:PERSON)
                    query = query_base + f"\n    SET e:ENTITY:{neo4j_label}\n    RETURN e"
                else:
                    # Solo usar label ENTITY
                    query = query_base + "\n    SET e:ENTITY\n    RETURN e"
                
                # Ejecutar query con parámetros (uri y name)
                session.run(query, uri=uri, name=name)
                
                entities_inserted += 1
                
                if entities_inserted % 100 == 0:
                    print(f"  Procesadas {entities_inserted} entidades...")

    print(f"✓ {entities_inserted} entidades insertadas")
    return entities_inserted

print("=" * 60)
print("INSERTANDO ENTIDADES")
print("=" * 60)
entities_count = insert_entities(driver, entities_file)


INSERTANDO ENTIDADES
  Procesadas 100 entidades...
  Procesadas 200 entidades...
  Procesadas 300 entidades...
  Procesadas 400 entidades...
  Procesadas 500 entidades...
  Procesadas 600 entidades...
  Procesadas 700 entidades...
  Procesadas 800 entidades...
  Procesadas 900 entidades...
  Procesadas 1000 entidades...
  Procesadas 1100 entidades...
  Procesadas 1200 entidades...
  Procesadas 1300 entidades...
  Procesadas 1400 entidades...
  Procesadas 1500 entidades...
  Procesadas 1600 entidades...
  Procesadas 1700 entidades...
  Procesadas 1800 entidades...
  Procesadas 1900 entidades...
  Procesadas 2000 entidades...
✓ 2084 entidades insertadas


## Paso 7: Insertar relaciones

Cargamos las relaciones desde el CSV y las conectamos entre las entidades existentes.


In [17]:
def insert_relations(driver, relations_file):
    """Inserta relaciones desde el CSV extraído del PDF"""
    if not relations_file or not os.path.exists(relations_file):
        return 0
    
    relations_inserted = 0
    
    # Template para crear relaciones
    create_relation_template = """
    MATCH (from:ENTITY {uri: $from_uri})
    MATCH (to:ENTITY {uri: $to_uri})
    MERGE (from)-[r:RELATED_TO {relation_type: $relation_type}]->(to)
    ON CREATE SET
        r.source = 'pdf',
        r.created_at = timestamp()
    ON MATCH SET
        r.updated_at = timestamp()
    RETURN r
    """.strip()

    # Template para relaciones con tipos específicos
    create_typed_relation_template = """
    MATCH (from:ENTITY {uri: $from_uri})
    MATCH (to:ENTITY {uri: $to_uri})
    MERGE (from)-[r:{relation_name}]->(to)
    ON CREATE SET
        r.source = 'pdf',
        r.relation_type = $relation_type,
        r.created_at = timestamp()
    ON MATCH SET
        r.updated_at = timestamp()
    RETURN r
    """.strip()

    with driver.session() as session:
        relations_processed = set()  # Para evitar duplicados
        
        with open(relations_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                subject = row.get('subject', '').strip()
                relation = row.get('relation', '').strip().lower()
                obj = row.get('object', '').strip()
                
                if not subject or not obj or len(subject) < 2 or len(obj) < 2:
                    continue
                
                # Generar URIs
                from_uri = generate_uri(subject)
                to_uri = generate_uri(obj)
                
                # Evitar duplicados
                relation_key = (from_uri, relation, to_uri)
                if relation_key in relations_processed:
                    continue
                relations_processed.add(relation_key)
                
                # Normalizar el tipo de relación
                relation_type = RELATION_TYPE_MAPPING.get(relation, relation.upper())
                
                # Determinar si usar relación genérica o específica
                if relation in RELATION_TYPE_MAPPING:
                    relation_name = RELATION_TYPE_MAPPING[relation].upper()
                    try:
                        session.run(create_typed_relation_template.format(relation_name=relation_name),
                                  from_uri=from_uri, to_uri=to_uri, relation_type=relation_type)
                    except Exception as e:
                        # Si falla, usar relación genérica
                        session.run(create_relation_template,
                                  from_uri=from_uri, to_uri=to_uri, relation_type=relation_type)
                else:
                    session.run(create_relation_template,
                              from_uri=from_uri, to_uri=to_uri, relation_type=relation_type)
                
                relations_inserted += 1
                
                if relations_inserted % 100 == 0:
                    print(f"  Procesadas {relations_inserted} relaciones...")

    print(f"✓ {relations_inserted} relaciones insertadas")
    return relations_inserted

print("\n" + "=" * 60)
print("INSERTANDO RELACIONES")
print("=" * 60)
if relations_file and os.path.exists(relations_file):
    relations_count = insert_relations(driver, relations_file)
else:
    print("⚠ No hay archivo de relaciones para cargar")
    relations_count = 0



INSERTANDO RELACIONES
  Procesadas 100 relaciones...
  Procesadas 200 relaciones...
  Procesadas 300 relaciones...
  Procesadas 400 relaciones...
  Procesadas 500 relaciones...
  Procesadas 600 relaciones...
  Procesadas 700 relaciones...
  Procesadas 800 relaciones...
  Procesadas 900 relaciones...
  Procesadas 1000 relaciones...
  Procesadas 1100 relaciones...
  Procesadas 1200 relaciones...
  Procesadas 1300 relaciones...
  Procesadas 1400 relaciones...
  Procesadas 1500 relaciones...
  Procesadas 1600 relaciones...
  Procesadas 1700 relaciones...
  Procesadas 1800 relaciones...
  Procesadas 1900 relaciones...
  Procesadas 2000 relaciones...
  Procesadas 2100 relaciones...
  Procesadas 2200 relaciones...
  Procesadas 2300 relaciones...
  Procesadas 2400 relaciones...
  Procesadas 2500 relaciones...
  Procesadas 2600 relaciones...
  Procesadas 2700 relaciones...
  Procesadas 2800 relaciones...
  Procesadas 2900 relaciones...
  Procesadas 3000 relaciones...
  Procesadas 3100 relacion

## Paso 8: Verificar resultados y resumen

Consultamos la base de datos para verificar que los datos se cargaron correctamente.


In [18]:
print("=" * 60)
print("VERIFICANDO RESULTADOS")
print("=" * 60)

with driver.session() as session:
    # Contar entidades del PDF
    result = session.run("MATCH (n:ENTITY) WHERE n.source = 'pdf' RETURN count(n) as count")
    total_entities = result.single()['count']
    
    # Contar relaciones del PDF
    result = session.run("MATCH ()-[r:RELATED_TO]->() WHERE r.source = 'pdf' RETURN count(r) as count")
    total_relations = result.single()['count']
    
    # Contar por tipo de entidad
    result = session.run("""
        MATCH (n:ENTITY) 
        WHERE n.source = 'pdf'
        RETURN labels(n) as labels, count(n) as count
        ORDER BY count DESC
        LIMIT 10
    """)
    print("\nEntidades por tipo (top 10):")
    for record in result:
        labels = [l for l in record['labels'] if l != 'ENTITY']
        label_str = ':'.join(labels) if labels else 'ENTITY'
        print(f"  {label_str}: {record['count']}")
    
    # Contar tipos de relaciones
    result = session.run("""
        MATCH ()-[r:RELATED_TO]->() 
        WHERE r.source = 'pdf'
        RETURN r.relation_type as type, count(r) as count
        ORDER BY count DESC
        LIMIT 10
    """)
    print("\nRelaciones por tipo (top 10):")
    for record in result:
        rel_type = record['type'] or 'RELATED_TO'
        print(f"  {rel_type}: {record['count']}")

print("\n" + "=" * 60)
print("RESUMEN FINAL")
print("=" * 60)
print(f"Entidades insertadas en esta sesión: {entities_count}")
print(f"Relaciones insertadas en esta sesión: {relations_count}")
print(f"\nTotal en la base de datos:")
print(f"  - Entidades del PDF: {total_entities}")
print(f"  - Relaciones del PDF: {total_relations}")

# Cerrar conexión
driver.close()
print("\n✓ Conexión cerrada")
print("✓ Proceso completado exitosamente")


VERIFICANDO RESULTADOS

Entidades por tipo (top 10):
  ENTITY: 1664
  ORG: 171
  LOC: 99
  PERSON: 53
  ORG:LOC: 7
  PERSON:LOC: 6
  PERSON:ORG: 2

Relaciones por tipo (top 10):
  DE DE: 948
  DE: 558
  •: 279
  EN DE: 270
  PARA PARA: 193
  DE CON: 178
  OFRECEN: 174
  PARA: 162
  PARA EN: 160
  CON CON: 152

RESUMEN FINAL
Entidades insertadas en esta sesión: 2084
Relaciones insertadas en esta sesión: 8623

Total en la base de datos:
  - Entidades del PDF: 2002
  - Relaciones del PDF: 8452

✓ Conexión cerrada
✓ Proceso completado exitosamente


## Paso 9: Visualizar el grafo (Opcional)

Puedes abrir el Neo4j Browser en http://localhost:7474 para visualizar el grafo.

### Consultas útiles para explorar:

```cypher
// Ver todas las entidades del PDF
MATCH (n:ENTITY) WHERE n.source = 'pdf' RETURN n LIMIT 25

// Ver relaciones del PDF
MATCH (a:ENTITY)-[r:RELATED_TO]->(b:ENTITY) 
WHERE r.source = 'pdf' 
RETURN a, r, b 
LIMIT 50

// Contar entidades por tipo
MATCH (n:ENTITY) WHERE n.source = 'pdf'
RETURN labels(n) as tipo, count(n) as cantidad
ORDER BY cantidad DESC

// Ver entidades más conectadas
MATCH (n:ENTITY)-[r:RELATED_TO]->() 
WHERE n.source = 'pdf'
RETURN n.name, count(r) as conexiones
ORDER BY conexiones DESC
LIMIT 10
```
